In [94]:
import pandas as pd
import numpy as np
import datetime
import pytz
import os

debug = False

In [95]:
if os.getenv("COLAB_RELEASE_TAG"):
    google_colab = True
    print("Running in Colab")
    folder_path = '/content/drive/MyDrive/stock'
    from google.colab import drive
    drive.mount('/content/drive')
else:
    google_colab = False
    print("NOT in Colab")
    folder_path = "."


NOT in Colab


In [96]:
# Modify the name of the picklist file as per actuals
transfer_file = f'{folder_path}/the-new-picklist.xlsx'
volume_file = f"{folder_path}/volumes.xlsx"
# modify result file, if required
current_time = datetime.datetime.now(pytz.timezone('Asia/Kolkata')) 
the_date = current_time.strftime("%Y-%m-%d-%H-%M-%S")

stock_file = f"{folder_path}/{the_date}-Stock-Status.xlsx"
debug_file = f"{folder_path}/{the_date}-Debug.xlsx"

In [97]:
opening_file_sheet = "opening"
transfer_file_sheet = "transfers"
picked_file_sheet = "picked"
placed_file_sheet = "placed"

almirah_file_sheet = "Almirahs"
book_file_sheet = "Books"

In [98]:
opening = pd.read_excel (transfer_file, sheet_name=opening_file_sheet, 
                              converters={'BookCode' : str, 'Number': int, 'FromAlmirah': str, 
                                                     'FromRack': int, 'ToAlmirah': str, 'ToRack':int,
                                                     'Comments' : str}, parse_dates = ['PickListDate'])
transfer = pd.read_excel (transfer_file, sheet_name=transfer_file_sheet, 
                          converters={'BookCode' : str, 'Number': int, 'FromAlmirah': str, 
                                                     'FromRack': int, 'ToAlmirah': str, 'ToRack':int,
                                                     'Comments' : str}, parse_dates = ['PickListDate'])
pickup = pd.read_excel (transfer_file, sheet_name=picked_file_sheet, 
                          converters={'BookCode' : str, 'Number': int, 'FromAlmirah': str, 
                                                     'FromRack': int, 'ToAlmirah': str, 'ToRack':int,
                                                     'Comments' : str}, parse_dates = ['PickListDate'])
deposit = pd.read_excel (transfer_file, sheet_name=placed_file_sheet, 
                          converters={'BookCode' : str, 'Number': int, 'FromAlmirah': str, 
                                                     'FromRack': int, 'ToAlmirah': str, 'ToRack':int,
                                                     'Comments' : str}, parse_dates = ['PickListDate'])
almirah_bins_dimensions = pd.read_excel (volume_file, sheet_name=almirah_file_sheet,
                                         converters={'Almirah' : str, 'RackNumber': int, 'Height': float, 
                                                     'Length': float, 'Depth': float})

almirah_bins_dimensions.rename(columns={"RackNumber": "Rack"}, inplace=True)
book_dimensions = pd.read_excel (volume_file, sheet_name=book_file_sheet, 
                                 converters={'BookCode' : str, 'Height': int, 'Length': float, 
                                             'Depth': float, 'Number': int})

In [99]:
almirah_bins_dimensions = almirah_bins_dimensions.filter(['Almirah', 'Rack', 'Height', 'Length', 'Depth'])
almirah_bins_dimensions['Volume'] = almirah_bins_dimensions['Height'] * almirah_bins_dimensions['Length'] * almirah_bins_dimensions['Depth']

In [100]:
n_opening = opening[['ToAlmirah', 'ToRack', 'BookCode', 'Number']].copy(deep=True)
n_opening.rename(columns = {'ToAlmirah':'Almirah', 'ToRack':'Rack'}, inplace = True)
n_opening.dropna(axis=0, how='any', subset=['Almirah', 'Rack'], inplace=True)

In [101]:
n_tpickup = transfer[['FromAlmirah', 'FromRack', 'BookCode', 'Number']].copy(deep=True)
n_tpickup['PickedNumber'] = - n_tpickup['Number']
n_tpickup.drop('Number', axis=1, inplace=True)
n_tpickup.rename(columns = {'FromAlmirah':'Almirah', 'FromRack':'Rack', 'PickedNumber' : 'Number'}, inplace = True)
n_tpickup.dropna(axis=0, how='any', subset=['Almirah', 'Rack'], inplace=True)

In [102]:
n_tdeposit = transfer[['ToAlmirah', 'ToRack', 'BookCode', 'Number']].copy(deep=True)
n_tdeposit.rename(columns = {'ToAlmirah':'Almirah', 'ToRack':'Rack'}, inplace = True)
n_tdeposit.dropna(axis=0, how='any', subset=['Almirah', 'Rack'], inplace=True)

In [103]:
n_pure_deposit = deposit[['ToAlmirah', 'ToRack', 'BookCode', 'Number']].copy(deep=True)
n_pure_deposit.rename(columns = {'ToAlmirah':'Almirah', 'ToRack':'Rack'}, inplace = True)
n_pure_deposit.dropna(axis=0, how='any', subset=['Almirah', 'Rack'], inplace=True)

In [104]:
n_pure_pickup = pickup[['FromAlmirah', 'FromRack', 'BookCode', 'Number']].copy(deep=True)
n_pure_pickup['PickedNumber'] = - n_pure_pickup['Number']
n_pure_pickup.drop('Number', axis=1, inplace=True)
n_pure_pickup.rename(columns = {'FromAlmirah':'Almirah', 'FromRack':'Rack', 'PickedNumber' : 'Number'}, 
                     inplace = True)
n_pure_pickup.dropna(axis=0, how='any', subset=['Almirah', 'Rack'], inplace=True)

In [105]:
updatedloc = pd.concat([n_opening, n_tpickup, n_tdeposit, n_pure_pickup, n_pure_deposit], ignore_index=True)

In [106]:
# this cell is only for debug purposes
if debug:
    with pd.ExcelWriter(debug_file) as writer:  
        n_opening.to_excel(writer, sheet_name='Opening')
        n_tpickup.to_excel(writer, sheet_name='TransferPickup')
        n_tdeposit.to_excel(writer, sheet_name='TransferDeposit')
        n_pure_deposit.to_excel(writer, sheet_name='PureDeposit')
        n_pure_pickup.to_excel(writer, sheet_name='PurePickup')
        updatedloc.to_excel(writer, sheet_name='UpdatedLoc')

In [107]:
updated_report = pd.pivot_table(updatedloc, values='Number', 
                                index=['Almirah', 'Rack', 'BookCode'], 
                                aggfunc=np.sum)
updated_report.reset_index(inplace=True)
updated_locations_filtered = updated_report[updated_report['Number'] != 0]

# Bookwise sorting
picklist_sheet = updated_locations_filtered.sort_values(by=['BookCode', 'Number', "Almirah", "Rack"])
# almirah wise sorting
almirah_sheet = updated_locations_filtered.sort_values(by=["Almirah", "Rack", 'BookCode', 'Number'])


In [108]:
stock_summary = pd.pivot_table(updated_locations_filtered, values='Number', index=['BookCode'], aggfunc=np.sum)
stock_summary.sort_values(by=['BookCode'], inplace=True)

In [109]:
volume_sheet = pd.merge(almirah_sheet, book_dimensions, how='left', on='BookCode')
volume_sheet['Volume'] = volume_sheet['Number_x'] * volume_sheet['Height'] *  volume_sheet['Width'] *  volume_sheet['Depth'] /  volume_sheet['Number_y'] 
volume_sheet = volume_sheet.filter(['Almirah', 'Rack', 'BookCode', 'Volume'])
volume_sheet.rename(columns = {'Volume':'OccupiedBookVolume'}, inplace = True)
volume_sheet_summary = pd.pivot_table(volume_sheet, values='OccupiedBookVolume', index=['Almirah', 'Rack'], aggfunc=np.sum)
volume_sheet_summary.reset_index(inplace=True)

In [110]:
volume_report = pd.merge(volume_sheet_summary, almirah_bins_dimensions, how='left', on=['Almirah', 'Rack'])
volume_report['FractionOccupied'] = volume_report['OccupiedBookVolume']/volume_report['Volume']
volume_report['AvailableVolume'] = volume_report['Volume'] - volume_report['OccupiedBookVolume']
volume_report.drop(columns=['Height', 'Length', 'Depth'])
volume_report.sort_values(by=["Almirah", "Rack"], inplace=True)
volume_report_by_available = volume_report.sort_values(by="AvailableVolume", ascending=False)

In [111]:
# Write down final results
with pd.ExcelWriter(stock_file) as writer:  
    almirah_sheet.to_excel(writer, sheet_name='AlmirahRack')
    picklist_sheet.to_excel(writer, sheet_name='PicklistMaker')
    stock_summary.to_excel(writer, sheet_name='StockSummary')
    volume_report.to_excel(writer, sheet_name='VolumeReport')
    volume_report_by_available.to_excel(writer, sheet_name='VolumeAvailable')
print(f"Result written to {stock_file}")

Result written to ./2023-09-09-17-10-33-Stock-Status.xlsx


In [112]:
if google_colab:
    drive.flush_and_unmount()